# Network Deliberation ABM
## Discrete state, generated signal task

In [ ]:
# Imports
import configparser
import math
import random

import networkx as nx
import numpy as np
from numpy import random as nprand
import repsci
from tqdm.notebook import tqdm

import netdelib.soclearn.evaluate as sleval
import netdelib.soclearn.models.generated as slgen
import netdelib.soclearn.strategy as slstrat
from netdelib.topologies.factories import *

from discrete import run_discrete_trial

In [ ]:
# Configuration
config = configparser.ConfigParser()
config.read('discrete_generated.cfg')

N = config.getint('abm', 'N')
M = config.getint('abm', 'M')
runs = config.getint('abm', 'runs')
stages = config.getint('abm', 'stages')
steps = config.getint('abm', 'steps')

bit_count = config.getint('abm', 'bit_count')
p_error = config.getfloat('abm', 'p_error')

small_world_k = config.getint('networks', 'small_world_k')
small_world_a = config.getfloat('networks', 'small_world_a')
barabasi_albert_m = config.getint('networks', 'barabasi_albert_m')

# Configure plotting in Jupyter
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams.update({
    'figure.figsize': (20, 20),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})

all_networks = {
    'Complete': CompleteFactory(N, M),
    'Pref. Attach.': PreferentialFactory(N, M, barabasi_albert_m),
    'Small World': SmallWorldFactory(N, M, small_world_k, small_world_a),
    'Long Path': LongPathFactory(N, M),
    'Random Group': RandomGroupFactory(N, M)   
}
networks = dict(
    (k, v) for k, v in all_networks.items()
    if k in config.get('networks', 'enabled').split(','))

all_learning_strategies = {
    'Best Neighbor': slstrat.best_neighbor,
    'Conform': slstrat.conform,
    'Local Majority': slstrat.local_majority,
}
learning_strategies = dict(
    (k, v) for k, v in all_learning_strategies.items()
    if k in config.get('strategies', 'enabled').split(','))


# Run Simulation 

In [ ]:
experiment = repsci.Experiment("discrete_generated", config=config)
log = experiment.get_logger()

num_net = len(networks)
num_strat = len(learning_strategies)
true_value = tuple([1 for bit in range(bit_count)])
score_fraction = [[0 for strat in range(num_net)] for net in range(num_strat)]
score_distance = [[0 for strat in range(num_net)] for net in range(num_strat)]

for i, (title, learning_strategy) in enumerate(tqdm(learning_strategies.items())):
    log.info('Running trials for strategy: {}'.format(title))
    
    for j, (title2, factory) in enumerate(networks.items()):
        log.info('....Running trials for network: {}'.format(title2))
        
        # Record time series for scores
        run_fraction = []
        run_distance = []
        for run in range(runs):
            initial_beliefs = slgen.initial_beliefs_noisy(N, true_value, p_error=p_error)
            objective = lambda belief: sum([1 for index, true_bit in enumerate(true_value) if belief[index] == true_bit])
            beliefs = run_discrete_trial(
                factory,
                learning_strategy,
                initial_beliefs,
                objective,
                N, M, stages, steps)
            run_fraction.append(sleval.beliefs_correct(beliefs, true_value))
            run_distance.append(sleval.belief_distance(beliefs, true_value))
        # Fraction correct
        mean = np.mean(run_fraction, axis=0)
        std = np.std(run_fraction, axis=0)
        score_fraction[i][j] = (mean, std)
        # Mean distance
        mean = np.mean(run_distance, axis=0)
        std = np.std(run_distance, axis=0)
        score_distance[i][j] = (mean, std)
        
log.info('All trials complete')

In [ ]:
out = experiment.get_filename('score_fraction.npy')
np.save(out, score_fraction)
out = experiment.get_filename('score_distance.npy')
np.save(out, score_distance)